## Check List 3.1 - MultiICP Detector

This test is for checking functionality of MultiICP. All data are prepared as file, no HW required.

* **3.1.1 MultiICP**  
  - initialize, set_config, detect, detect_and_register, disconnect  
  
  
* **TBD**
  - Auto initialization to estimate initial guess for ICP is not perfect
  - Robust and reliable initial guess for gloabl registration will be done
  - Multiple instance for the same class will be done

## Set running directory to Project source

In [1]:
import os
import sys
import numpy as np
import cv2
import copy
import matplotlib.pyplot as plt
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.geometry.geometry import GeometryItem
from pkg.geometry.geotype import GEOTYPE
from pkg.detector.detector_interface import DetectionLevel
from pkg.detector.multiICP.config import *

In [3]:
# add camera geometry
def add_cam(gscene, tool_link="indy0_tcp"):
    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam", link_name=tool_link,
                       dims=(0.061, 0.061, 0.026), center=(-0.0785, 0, 0.013), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.5), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam_col", link_name=tool_link,
                       dims=(0.081, 0.081, 0.046), center=(-0.0785, 0, 0.013), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)

    viewpoint = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="viewpoint", link_name=tool_link,
                                   dims=(0.01, 0.01, 0.01), center=(-0.013, 0, 0), rpy=(0, 0, -np.pi / 2),
                                   color=(1, 0, 0, 0.3), display=True, fixed=True, collision=False, parent="cam")

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body", link_name=tool_link,
                       dims=(0.067, 0.067, 0.0335), center=(-0.0785, 0, -0.01675), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 1), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body_col", link_name=tool_link,
                       dims=(0.087, 0.087, 0.0535), center=(-0.0785, 0, -0.01675), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)

    gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead", link_name=tool_link,
                       dims=(0.067, 0.067, 0.067), center=(-0.0785, 0, -0.0335), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 1), display=True, fixed=True, collision=False)

    gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead_col", link_name=tool_link,
                       dims=(0.087, 0.087, 0.087), center=(-0.0785, 0, -0.0335), rpy=(0, 0, 0),
                       color=(0.8, 0.8, 0.8, 0.2), display=True, fixed=True, collision=True)
    return viewpoint

### 3.1.1 MultiICP

In [4]:
from pkg.detector.multiICP.multiICP import MultiICP, MultiICP_Obj
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

##### create multiICP instance

In [5]:
micp = MultiICP(None)

##### initialize()

In [6]:
config_list, img_dim = load_pickle(RNB_PLANNING_DIR+"release/multiICP_data/cam_configs.pkl")
micp.initialize(config_list, img_dim)

Camera is not set - skip initialization, use manually given camera configs


##### create SceneBuilder instance and geometry scene

In [7]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(None, None))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                           None, root_on="kmb0_platform", 
                           specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, False])

scene_builder = SceneBuilder.instance(detector=micp)
gscene = scene_builder.create_gscene(crob)

from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
VIEW_POSE = np.deg2rad([  0., 50.,  -70.,  -0.,  -90., 0])
VIEW_LOC = [0]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
T_bc = viewpoint.get_tf(VIEW_POSE_EXT)
T_cb = SE3_inv(T_bc)
gscene.show_pose(VIEW_POSE_EXT)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Set MultiICP configs
* You have to make micp, hrule and grule for each object you want to detect

* hrule means heuristic rule for special object which cannot be detected thorugh mask rcnn using COCO dataset

* grule means initial guess(R,t) for ICP

* Run shared detector to detect object in color image

##### Run shared detector for object detection on bash
* To skip detector initiailzation, run shared detector on separate terminal as follows
* set proper values to dims: desired size of MultiICP = reversed(micp.dsize)+(3,)
```bash
python3 $RNG_PLANNING_DIR/src/pkg/detector/multiICP/shared_detector.py --dims='(720,1280,3)'
```


#### Clearing shared detector channels if zombie memory remains
```python
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")
```

In [8]:
from pkg.utils.shared_function import clear_channels_on, sa
clear_channels_on("SharedDetector")

from pkg.detector.multiICP.shared_detector import SharedDetectorGen
sd = SharedDetectorGen(tuple(reversed(micp.dsize))+(3,))()
sd.init()

##### Load example data (for bed, closet)
* You need to prepare example data and stl model to test MultiICP detector 
* Use color image, depth image and csv file which has joint values Q with cam_intrins, depth_scale 
* file path: release/multiICP_data/

In [10]:
def load_rdict(file_name):
    rdict = {}
    
    rdict['color'] = cv2.imread(
        os.path.join('../release/multiICP_data/', file_name + '.jpg'), flags=cv2.IMREAD_UNCHANGED)
    rdict['depth'] = cv2.imread(
        os.path.join('../release/multiICP_data/', file_name + '.png'), flags=cv2.IMREAD_UNCHANGED)
    
    Q = np.loadtxt(os.path.join('../release/multiICP_data/', file_name + '.csvd'), delimiter=",")
    return rdict, np.array(Q)


rdict, Qtest = load_rdict("test_1")
color_img = rdict['color']
depth_img = rdict['depth']
Tc = viewpoint.get_tf(Qtest)
gscene.show_pose(Qtest)

In [11]:
T_cb = SE3_inv(Tc)

In [12]:
obj_info_dict = get_obj_info()

In [13]:
# object items you want to detect
# heuristic rule items, Initial guess rule items
micp_bed = MultiICP_Obj(obj_info_dict["bed"], None,
                        OffsetOnModelCoord("bed", R=np.matmul(T_cb[:3, :3], Rot_axis(3, np.pi)),
                                          offset=np.matmul(T_cb[:3, :3], (1.1 * 0.5, 0, -0.5))))

mrule_closet = MaskBoxRule("closet", "bed", merge_rule=np.all)
mrule_closet.update_rule = ClosetRuleFun(mrule_closet)
micp_closet = MultiICP_Obj(obj_info_dict["closet"], 
                           mrule_closet,
                           OffsetOnModelCoord("closet", 
                                             offset=(0, 1, 0.3),
                                             use_median=True
                                     ))

In [14]:
micp_dict = {"bed": micp_bed, "closet": micp_closet}

In [15]:
micp.set_config(micp_dict, sd, crob, viewpoint)

##### detect()
* Object Detection through swin-transformer based mask rcnn using mmdet
* To use test data(color, depth image and joint value Q), you have to call cache_sensor()
* Detection result is the transformation w.r.t base coord

In [17]:
micp.cache_sensor(color_img, depth_img, Qtest)

In [18]:
pose_dict = micp.detect(visualize=True)

name_mask is None
===== Detected : handbag, 1 object(s) =====
===== Detected : suitcase, 1 object(s) =====
===== Detected : bowl, 3 object(s) =====
===== Detected : bed, 1 object(s) =====
===== Detected : clock, 1 object(s) =====

'bed' is not in gscene. Use manual input for initial guess

Apply point-to-point ICP
registration::RegistrationResult with fitness=8.364424e-01, inlier_rmse=6.819923e-02, and correspondence_set size of 5022
Access transformation to get result.
Transformation is:
[[ 0.82499418 -0.10027313  0.55617438 -0.58251586]
 [-0.22097802 -0.96301758  0.15416169  0.65528698]
 [ 0.52014745 -0.25008479 -0.81664205  4.63277458]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=8.555687e-01, inlier_rmse=4.215075e-02, and correspondence_set size of 2174
Access transformation to get result.
Transformation is:
[[ 0.82574995 -0.06843705  0.55986911 -0.56611926]
 [-0.15982027 -0.98032022  0.11588664  0.724683

##### detect_and_register()
* Detect items in the field of view and register them to the GeometryScene
* They will appear in the RVIZ



In [20]:
micp.cache_sensor(color_img, depth_img, Qtest)

In [21]:
gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
# gtem_dict = scene_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

name_mask is ['closet', 'bed']
===== Detected : bed, 1 object(s) =====

'bed' is not in gscene. Use manual input for initial guess

Apply point-to-point ICP
registration::RegistrationResult with fitness=8.306129e-01, inlier_rmse=6.825225e-02, and correspondence_set size of 4987
Access transformation to get result.
Transformation is:
[[ 0.8230461  -0.09851426  0.5593658  -0.58957713]
 [-0.22039279 -0.96307013  0.15467031  0.65483328]
 [ 0.52347127 -0.25058097 -0.81436294  4.62936601]
 [ 0.          0.          0.          1.        ]]
Apply point-to-point ICP
registration::RegistrationResult with fitness=8.402506e-01, inlier_rmse=4.257531e-02, and correspondence_set size of 2146
Access transformation to get result.
Transformation is:
[[ 0.82494329 -0.0577468   0.56225787 -0.57826874]
 [-0.15136579 -0.98100334  0.12132935  0.71698996]
 [ 0.54457048 -0.18519643 -0.81801301  4.61795167]
 [ 0.          0.          0.          1.        ]]
Found 6DoF pose of bed
===== Apply heuristic rule fo

##### disconnect()

In [22]:
micp.disconnect()